In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [2]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models
bert_path = "../../corpus/pretrained/sbert_snli"
sbert = SentenceTransformer(bert_path)

[677] 2022-01-11 00:37:13,677 Info sentence_transformers.SentenceTransformer :Load pretrained SentenceTransformer: ../../corpus/pretrained/sbert_snli
[677] 2022-01-11 00:37:14,780 Info sentence_transformers.SentenceTransformer :Use pytorch device: cuda


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim
import torch.nn.utils.rnn as rnn

In [4]:
def text2vec(text):
    sentences = []
    if isinstance(text, str):
        sentences = [text]
    elif isinstance(text, list):
        sentences = text
    
    return sbert.encode(sentences)

def text2feature(text):
    vector = text2vec(text)
    diff = np.abs( vector[0] - vector[1] )
    return np.concatenate([vector.flatten(), diff])


def vec2feature(vector):
    diff = np.abs( vector[0] - vector[1] )
    return np.concatenate([vector.flatten(), diff])

In [5]:
class SNLIModel(nn.Module):
    def __init__(self, embedding_dim, tagset_size):
        # 親クラスのコンストラクタ。決まり文句
        super(SNLIModel, self).__init__()    
        self.embedding_dim = embedding_dim
        self.hid1= embedding_dim*2
        self.hid2 = embedding_dim//2
        self.fc1 = nn.Linear(self.embedding_dim, self.hid1)
        self.fc2 = nn.Linear(self.hid1, self.hid2)
        # LSTMの出力を受け取って全結合してsoftmaxに食わせるための１層のネットワーク
        # self.hidden2tag = nn.Linear(self.hid2+self.fb_dim, tagset_size)
        self.hidden2tag = nn.Linear(self.hid2, tagset_size)
    
    def forward(self, x):
        # fb = x[:, :self.fb_dim]
        y = F.relu(self.fc1(x))
        # y = F.relu(self.fc1(x[]))
        y = F.relu(self.fc2(y))
        y = self.hidden2tag( y )
        y = F.log_softmax(y, dim=1)
        return y

In [6]:
model_path = "../models/context/"
model_name = "sbert_snli_dnn.pickle"
modelM = DataManager(model_path)
model = modelM.load_data(model_name)

success load : ../models/context/sbert_snli_dnn.pickle


In [19]:
symbol = {"unk_token": "[UNK]", "sep_token": "[SEP]", "pad_token": "[PAD]", "cls_token": "[CLS]", "mask_token": "[MASK]"}

In [20]:
# path = "../../corpus/eval_labeled/"
path = "../eval_labeled/"

datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [32]:
def add_prev_utt(dialogue:list):
    base = ""
    added = []
    for utt in dialogue:
        if base=="":
            added.append(utt)
        else:
            added.append(base+symbol["sep_token"]+utt)
        base = utt
    # 巻き戻す
    added[-1] = dialogue[-1]
    return added

def make_Xy(convs, N=7):
    errors = ["Contradiction", "Self-contradiction"]
    print(errors)
    X = []
    y = []
    for conv in convs:
        dialogue = [""]*N
        for i, ut in enumerate( conv ) :
            # ユーザ発話駆動
            dialogue.append(clean_text( ut.utt) )
            if ut.is_system() and ut.is_exist_error():
                X.append( add_prev_utt( dialogue[-N:] ))
                # X.append(dialogue[-N:])
                if ut.is_error_included(errors) :
                    y.append(1)
                else:
                    y.append(0)
    
    return X, y

In [33]:
N=6
X_str, y = make_Xy(convs, N=N)

['Contradiction', 'Self-contradiction']


In [34]:
X_str

[['',
  '',
  '',
  'こんにちは。気軽に声かけてね。',
  'こんにちは。気軽に声かけてね。[SEP]こんにちは。はじめまして。',
  'お、そうですか。えっと、四国は行かれたことありますか?'],
 ['',
  'こんにちは。気軽に声かけてね。',
  'こんにちは。気軽に声かけてね。[SEP]こんにちは。はじめまして。',
  'こんにちは。はじめまして。[SEP]お、そうですか。えっと、四国は行かれたことありますか?',
  'お、そうですか。えっと、四国は行かれたことありますか?[SEP]四国は行ったことありません。あなたは行かれたことありますか?',
  'お遍路の逆回りは迷いやすくて大変な分、御利益が大きいらしいよ。'],
 ['屋台の食べ物は美味しく感じますよね。食べ物は何が好きですか?',
  '屋台の食べ物は美味しく感じますよね。食べ物は何が好きですか?[SEP]好きな食べ物は、蕎麦かな。',
  '好きな食べ物は、蕎麦かな。[SEP]お蕎麦いいですね。麺類は私も好きですよ。',
  'お蕎麦いいですね。麺類は私も好きですよ。[SEP]兄は無類の麺類好きで、毎日ラーメンを昼食に食べているらしい。',
  '兄は無類の麺類好きで、毎日ラーメンを昼食に食べているらしい。[SEP]ラーメンもいいですね。お兄さんがいらっしゃるんですか?',
  'お兄さんに生まれると、頼り甲斐のある性格になるのかな。'],
 ['',
  '',
  '',
  'こんにちは。ちょうど退屈してたんだ。',
  'こんにちは。ちょうど退屈してたんだ。[SEP]こんにちは。元気ですか?',
  '元気ですかは元気です'],
 ['',
  'こんにちは。ちょうど退屈してたんだ。',
  'こんにちは。ちょうど退屈してたんだ。[SEP]こんにちは。元気ですか?',
  'こんにちは。元気ですか?[SEP]元気ですかは元気です',
  '元気ですかは元気です[SEP]元気そうでよかった。',
  '元気ですかは元気ですね'],
 ['こんにちは。元気ですか?',
  'こんにちは。元気ですか?[SEP]元気ですかは元気です',
  '元気ですかは元気です[SEP]元気そうでよかった。',
  '元気そうでよかった。[SEP]元気ですかは元気ですね',
 

In [35]:
X_str_all = sum(X_str, [])

In [36]:
len( X_str_all ) 

8316

In [37]:

X_all = text2vec(X_str_all)

Batches:   0%|          | 0/260 [00:00<?, ?it/s]

In [38]:
X_vec =  X_all.reshape(len(X_str), N, -1) 

In [39]:
X_vec.shape

(1386, 6, 768)

In [43]:
# 推定の時間だ
y_pred = []
for X in tqdm( X_vec):
    hypo = X[-1]
    is_contradict = False
    features = []
    for prev in X[:-1]:
        feature = vec2feature( np.array([prev, hypo]) )
        features.append(feature)
    features = np.array(features)
    with torch.no_grad():
        features = torch.from_numpy(features).cuda()
        y_pred_ = np.array(model(features).cpu()).argmax(axis=1)
    if 2 in y_pred_:
        y_pred.append(1)
    else:
        y_pred.append(0)
   

100%|██████████| 1386/1386 [00:02<00:00, 687.57it/s]


In [44]:
score(y, y_pred)

confusion matrix = 
 [[1208  156]
 [  17    5]]
accuracy =  0.8751803751803752
precision =  0.031055900621118012
recall =  0.22727272727272727
f1 score =  0.054644808743169404


- N=7 eval

        confusion matrix = 
        [[1166  198]
        [  16    6]]
        accuracy =  0.8455988455988456
        precision =  0.029411764705882353
        recall =  0.2727272727272727
        f1 score =  0.05309734513274336

    - 6件だけの検出
    - 結構ミスが多いので，もっとNを減らして効果を見る

- N=6 eval

        confusion matrix = 
        [[1173  191]
        [  16    6]]
        accuracy =  0.8506493506493507
        precision =  0.030456852791878174
        recall =  0.2727272727272727
        f1 score =  0.05479452054794521

    - おそらく最もここが精度が高い，が...

- N=4 eval

        confusion matrix = 
        [[1208  156]
        [  17    5]]
        accuracy =  0.8751803751803752
        precision =  0.031055900621118012
        recall =  0.22727272727272727
        f1 score =  0.054644808743169404

## 発話加算スタイル
- ### 各前提に，直前の発話を加算
- N=6 added

        confusion matrix = 
        [[1256  108]
        [  18    4]]
        accuracy =  0.9090909090909091
        precision =  0.03571428571428571
        recall =  0.18181818181818182
        f1 score =  0.05970149253731343
   
   - 結構誤検出が減る

- N=4 added

        confusion matrix = 
        [[1283   81]
        [  18    4]]
        accuracy =  0.9285714285714286
        precision =  0.047058823529411764
        recall =  0.18181818181818182
        f1 score =  0.07476635514018691

    - 減ったけどなぁ

- N=3 added

    confusion matrix = 
    [[1311   53]
    [  21    1]]
    accuracy =  0.9466089466089466
    precision =  0.018518518518518517
    recall =  0.045454545454545456
    f1 score =  0.02631578947368421

    - N=3, N=4で大きな違いはありそう
    - しかし，未検出の18を拾いたい

In [45]:
# 検出出来ているものはなんだ
i = 0
for i, x in enumerate(X_str):
    if y[i]==1 and y_pred[i]==0:
        print(x)

['ビーチの場所取りが必要なんだよね。', 'ビーチの場所取りが必要なんだよね。[SEP]ビーチに行きたいです??', 'ビーチに行きたいです??[SEP]行きたいね。なかなか時間が取れないんだけどね。', '行きたいね。なかなか時間が取れないんだけどね。[SEP]服をえらんだりするのはすきですか?', '服をえらんだりするのはすきですか?[SEP]センスが良くないから、同じような服ばかり選んでしまうよ。', 'センスはいいですね']
['仲間だ', '仲間だ[SEP]ナカーマ', 'ナカーマ[SEP]仲間由紀恵', '仲間由紀恵[SEP]仲間由紀恵は怖いですね', '仲間由紀恵は怖いですね[SEP]何考えてるか分からない感じある', '仲間由紀恵は美しいですね']
['ボートの使える所もありますね。', 'ボートの使える所もありますね。[SEP]活性が高いですね', '活性が高いですね[SEP]どのような活性ですか?', 'どのような活性ですか?[SEP]活性は高いです', '活性は高いです[SEP]活発なのはいいことです。', '活性が全体的に低いです']
['', '', '', 'こんにちは。熱中症に気をつけて。', 'こんにちは。熱中症に気をつけて。[SEP]はい。ありがとう。あなたも気を付けて。', '熱中症に気をつけないんですか?']
['', 'こんにちは。熱中症に気をつけて。', 'こんにちは。熱中症に気をつけて。[SEP]はい。ありがとう。あなたも気を付けて。', 'はい。ありがとう。あなたも気を付けて。[SEP]熱中症に気をつけないんですか?', '熱中症に気をつけないんですか?[SEP]小まめに水を飲んだりして、気を付けていますよ。', '熱中症に気をつけたいんでしょう?']
['はい。ありがとう。あなたも気を付けて。', 'はい。ありがとう。あなたも気を付けて。[SEP]熱中症に気をつけないんですか?', '熱中症に気をつけないんですか?[SEP]小まめに水を飲んだりして、気を付けていますよ。', '小まめに水を飲んだりして、気を付けていますよ。[SEP]熱中症に気をつけたいんでしょう?', '熱中症に気をつけたいんでしょう?[SEP]そうですね。0度倒れていますから、十分気を付けています。', '熱中症に気をつけなければなりませんでしたっ